In [2]:
from typing import Optional, List

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

from functools import partial

from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models import create_model
from timm.models.helpers import load_pretrained
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model
import torch.optim as optim
from einops.layers.torch import Rearrange

import pickle
import torch.utils.checkpoint as checkpoint
import numpy as np
import time
import os


# Model definition

In [3]:

def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'mean': IMAGENET_DEFAULT_MEAN, 'std': IMAGENET_DEFAULT_STD,
        'first_conv': 'patch_embed.proj', 'classifier': 'head',
        **kwargs
    }


default_cfgs = {
    'cswin_224': _cfg(),
    'cswin_384': _cfg(
        crop_pct=1.0
    ),

}


class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


class LePEAttention(nn.Module):
    def __init__(self, dim, resolution, idx, split_size=7, dim_out=None, num_heads=8, attn_drop=0., proj_drop=0.,
                 qk_scale=None):
        super().__init__()
        self.dim = dim
        self.dim_out = dim_out or dim
        self.resolution = resolution
        self.split_size = split_size
        self.num_heads = num_heads
        head_dim = dim // num_heads
        # NOTE scale factor was wrong in my original version, can set manually to be compat with prev weights
        self.scale = qk_scale or head_dim ** -0.5
        if idx == -1:
            H_sp, W_sp = self.resolution, self.resolution
        elif idx == 0:
            H_sp, W_sp = self.resolution, self.split_size
        elif idx == 1:
            W_sp, H_sp = self.resolution, self.split_size
        else:
            print("ERROR MODE", idx)
            exit(0)
        self.H_sp = H_sp
        self.W_sp = W_sp
        stride = 1
        self.get_v = nn.Conv2d(dim, dim, kernel_size=3, stride=1, padding=1, groups=dim)

        self.attn_drop = nn.Dropout(attn_drop)

        # self.out = None

    def im2cswin(self, x):
        B, N, C = x.shape
        H = W = int(np.sqrt(N))
        x = x.transpose(-2, -1).reshape(B, C, H, W)
        x = img2windows(x, self.H_sp, self.W_sp)
        x = x.reshape(-1, self.H_sp * self.W_sp, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3).contiguous()
        return x

    def get_lepe(self, x, func):
        B, N, C = x.shape
        H = W = int(np.sqrt(N))
        x = x.transpose(-2, -1).reshape(B, C, H, W)

        H_sp, W_sp = self.H_sp, self.W_sp
        x = x.reshape(B, C, H // H_sp, H_sp, W // W_sp, W_sp)
        x = x.permute(0, 2, 4, 1, 3, 5).reshape(-1, C, H_sp, W_sp)  ### B', C, H', W'

        lepe = func(x)  ### B', C, H', W'
        lepe = lepe.reshape(-1, self.num_heads, C // self.num_heads, H_sp * W_sp).permute(0, 1, 3, 2)

        x = x.reshape(-1, self.num_heads, C // self.num_heads, self.H_sp * self.W_sp).permute(0, 1, 3, 2)
        return x, lepe

    def forward(self, qkv):
        """
        x: B L C
        """
        q, k, v = qkv[0], qkv[1], qkv[2]

        ### Img2Window
        H = W = self.resolution
        B, L, C = q.shape
        assert L == H * W, "flatten img_tokens has wrong size"

        q = self.im2cswin(q)
        k = self.im2cswin(k)
        v, lepe = self.get_lepe(v, self.get_v)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))  # B head N C @ B head C N --> B head N N
        attn = nn.functional.softmax(attn, dim=-1, dtype=attn.dtype)
        attn = self.attn_drop(attn)

        x = (attn @ v) + lepe
        x = x.transpose(1, 2).reshape(-1, self.H_sp * self.W_sp, C)  # B head N N @ B head N C

        ### Window2Img
        x = windows2img(x, self.H_sp, self.W_sp, H, W).reshape(B, -1, C)  # B H' W' C
        # self.out = x.detach().clone()#.cpu()

        return x


class CSWinBlock(nn.Module):

    def __init__(self, dim, reso, num_heads,
                 split_size=7, mlp_ratio=4., qkv_bias=False, qk_scale=None,
                 drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm,
                 last_stage=False):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.patches_resolution = reso
        self.split_size = split_size
        self.mlp_ratio = mlp_ratio
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.norm1 = norm_layer(dim)

        if self.patches_resolution == split_size:
            last_stage = True
        if last_stage:
            self.branch_num = 1
        else:
            self.branch_num = 2
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(drop)

        if last_stage:
            self.attns = nn.ModuleList([
                LePEAttention(
                    dim, resolution=self.patches_resolution, idx=-1,
                    split_size=split_size, num_heads=num_heads, dim_out=dim,
                    qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
                for i in range(self.branch_num)])
        else:
            self.attns = nn.ModuleList([
                LePEAttention(
                    dim // 2, resolution=self.patches_resolution, idx=i,
                    split_size=split_size, num_heads=num_heads // 2, dim_out=dim // 2,
                    qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
                for i in range(self.branch_num)])

        mlp_hidden_dim = int(dim * mlp_ratio)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, out_features=dim, act_layer=act_layer,
                       drop=drop)
        self.norm2 = norm_layer(dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """

        H = W = self.patches_resolution
        B, L, C = x.shape
        assert L == H * W, "flatten img_tokens has wrong size"
        img = self.norm1(x)
        qkv = self.qkv(img).reshape(B, -1, 3, C).permute(2, 0, 1, 3)

        if self.branch_num == 2:
            x1 = self.attns[0](qkv[:, :, :, :C // 2])
            x2 = self.attns[1](qkv[:, :, :, C // 2:])
            attened_x = torch.cat([x1, x2], dim=2)
        else:
            attened_x = self.attns[0](qkv)
        attened_x = self.proj(attened_x)
        x = x + self.drop_path(attened_x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


def img2windows(img, H_sp, W_sp):
    """
    img: B C H W
    """
    B, C, H, W = img.shape
    img_reshape = img.reshape(B, C, H // H_sp, H_sp, W // W_sp, W_sp)
    img_perm = img_reshape.permute(0, 2, 4, 3, 5, 1).reshape(-1, H_sp * W_sp, C)
    return img_perm


def windows2img(img_splits_hw, H_sp, W_sp, H, W):
    """
    img_splits_hw: B' H W C
    """
    B = int(img_splits_hw.shape[0] / (H * W / H_sp / W_sp))

    img = img_splits_hw.reshape(B, H // H_sp, W // W_sp, H_sp, W_sp, -1)
    img = img.permute(0, 1, 3, 2, 4, 5).reshape(B, H, W, -1)
    return img


class Merge_Block(nn.Module):
    def __init__(self, dim, dim_out, norm_layer=nn.LayerNorm):
        super().__init__()
        self.conv = nn.Conv2d(dim, dim_out, 3, 2, 1)
        self.norm = norm_layer(dim_out)

    def forward(self, x):
        B, new_HW, C = x.shape
        H = W = int(np.sqrt(new_HW))
        x = x.transpose(-2, -1).reshape(B, C, H, W)
        x = self.conv(x)
        B, C = x.shape[:2]
        x = x.reshape(B, C, -1).transpose(-2, -1)
        x = self.norm(x)

        return x


class CSWinTransformer(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=[1000], embed_dim=96, depth=[2, 2, 6, 2],
                 split_size=[3, 5, 7],
                 num_heads=12, mlp_ratio=4., qkv_bias=True, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., hybrid_backbone=None, norm_layer=nn.LayerNorm, use_chk=False,
                 circle_loss_enabled=False, normed_linear=False, keep_norm=False, aggregate=False):
        super().__init__()
        self.use_chk = use_chk
        self.head_class_counts = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.keep_norm = keep_norm
        self.aggregate = aggregate
        heads = num_heads

        self.stage1_conv_embed = nn.Sequential(
            nn.Conv2d(in_chans, embed_dim, 7, 4, 2),
            Rearrange('b c h w -> b (h w) c', h=img_size // 4, w=img_size // 4),
            nn.LayerNorm(embed_dim)
        )

        curr_dim = embed_dim
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, np.sum(depth))]  # stochastic depth decay rule
        self.stage1 = nn.ModuleList([
            CSWinBlock(
                dim=curr_dim, num_heads=heads[0], reso=img_size // 4, mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias, qk_scale=qk_scale, split_size=split_size[0],
                drop=drop_rate, attn_drop=attn_drop_rate,
                drop_path=dpr[i], norm_layer=norm_layer)
            for i in range(depth[0])])

        self.merge1 = Merge_Block(curr_dim, curr_dim * 2)
        curr_dim = curr_dim * 2
        self.stage2 = nn.ModuleList(
            [CSWinBlock(
                dim=curr_dim, num_heads=heads[1], reso=img_size // 8, mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias, qk_scale=qk_scale, split_size=split_size[1],
                drop=drop_rate, attn_drop=attn_drop_rate,
                drop_path=dpr[np.sum(depth[:1]) + i], norm_layer=norm_layer)
                for i in range(depth[1])])

        self.merge2 = Merge_Block(curr_dim, curr_dim * 2)

        curr_dim = curr_dim * 2

        self.stage3 = nn.ModuleList([CSWinBlock(
            dim=curr_dim, num_heads=heads[2], reso=img_size // 16, mlp_ratio=mlp_ratio,
            qkv_bias=qkv_bias, qk_scale=qk_scale, split_size=split_size[2],
            drop=drop_rate, attn_drop=attn_drop_rate,
            drop_path=dpr[np.sum(depth[:2]) + i], norm_layer=norm_layer)
            for i in range(depth[2])])

        self.final_heads = nn.ModuleList([
            nn.ModuleList([
                Merge_Block(curr_dim, curr_dim * 2),
                nn.Sequential(
                    *[CSWinBlock(
                        dim=curr_dim * 2, num_heads=heads[3], reso=img_size // 32, mlp_ratio=mlp_ratio,
                        qkv_bias=qkv_bias, qk_scale=qk_scale, split_size=split_size[-1],
                        drop=drop_rate, attn_drop=attn_drop_rate,
                        drop_path=dpr[np.sum(depth[:-1]) + i], norm_layer=norm_layer, last_stage=True)
                        for i in range(depth[-1])]),
                norm_layer(curr_dim * 2),
                nn.Linear(curr_dim * 2, head_class_count)
            ])
            for head_class_count in self.head_class_counts])

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, (nn.LayerNorm, nn.BatchNorm2d)):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'pos_embed', 'cls_token'}

    def get_classifier(self):
        return self.head

    def reset_classifier(self, num_classes, global_pool=''):
        if self.num_classes != num_classes:
            print('reset head to', num_classes)
            self.num_classes = num_classes
            self.head = nn.Linear(self.out_dim, num_classes) if num_classes > 0 else nn.Identity()
            self.head = self.head.cuda()
            trunc_normal_(self.head.weight, std=.02)
            if self.head.bias is not None:
                nn.init.constant_(self.head.bias, 0)

    def forward_backbone(self, x):
        for pre, blocks in zip([self.merge1, self.merge2],
                               [self.stage2, self.stage3]):
            x = pre(x)
            for blk in blocks:
                if self.use_chk:
                    x = checkpoint.checkpoint(blk, x)
                else:
                    x = blk(x)
        return x

    def forward_features(self, x):
        B = x.shape[0]
        x = self.stage1_conv_embed(x)

        for blk in self.stage1:
            if self.use_chk:
                x = checkpoint.checkpoint(blk, x)
            else:
                x = blk(x)

        x = self.forward_backbone(x)

        results = []
        for head_idx, head_layer in enumerate(self.final_heads):
            merge, stage, norm, head = head_layer

            head_x = merge(x)
            head_x = stage(head_x)
            head_x = norm(head_x)
            head_x = torch.mean(head_x, dim=1)
            head_x = head(head_x)
            results += [head_x]

        return results

    def forward(self, x):
        x = self.forward_features(x)
        return x

    def get_last_shared_params(self):
        return self.stage3.parameters()


def _conv_filter(state_dict, patch_size=16):
    """ convert patch embedding weight from manual patchify + linear proj to conv"""
    out_dict = {}
    for k, v in state_dict.items():
        if 'patch_embed.proj.weight' in k:
            v = v.reshape((v.shape[0], 3, patch_size, patch_size))
        out_dict[k] = v
    return out_dict


### 224 models


@register_model
def cswin_tiny_224_min(pretrained=False, pretrained_cfg='', head_class_counts=[10, 10], bn_tf=False, **kwargs):
    model = CSWinTransformer(patch_size=4, embed_dim=64, depth=[1, 2, 21, 1],
                             split_size=[1, 2, 7, 7], num_heads=[2, 4, 8, 16], mlp_ratio=4.,
                             num_classes=head_class_counts, **kwargs)
    model.default_cfg = default_cfgs['cswin_224']
    return model




# Env setup

In [5]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [15]:
batch_size = 2
num_epochs = 10
alpha = 1.5
head_class_counts = [10, 10]
num_tasks = 2
# num_tasks = len(head_class_counts)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_chk = True

In [16]:
model = create_model('cswin_tiny_224_min', head_class_counts=head_class_counts, use_chk=use_chk).to(device)
lambda_weights = torch.ones((num_tasks, ), requires_grad=True, device=device)

In [9]:
loss_fn = nn.CrossEntropyLoss().to(device)
model_optim = optim.AdamW(model.parameters(), lr=0.01)
lambda_optim = optim.AdamW([lambda_weights])


In [10]:
transform = transforms.Compose([
    transforms.Resize(size=224),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load the MNIST training set
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_dataset = Subset(train_dataset, range(300)) # just for the example to get faster results

# Create a DataLoader for the training set
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Download and load the MNIST test set
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_dataset = Subset(test_dataset, range(300))

# Create a DataLoader for the test set
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


# Train

In [17]:
initial_loss = None


for epoch in range(num_epochs):
    running_loss_a = 0.0
    running_loss_b = 0.0
    correct_predictions = 0
    total_samples = 0
    loss_m = AverageMeter()
    loss_a_m = AverageMeter()
    loss_b_m = AverageMeter()
    weights_a_m = AverageMeter()
    weights_b_m = AverageMeter()
    model.train()

    for idx, (img, target) in enumerate(train_dataloader):
        img = img.to(device)
        target = target.to(device)
        target_b = torch.flip(target, [0])

        model_optim.zero_grad()
        lambda_optim.zero_grad()

        out_a, out_b= model(img)

        loss_a = loss_fn(out_a, target)
        loss_b = loss_fn(out_b, target_b)

        loss_tasks = torch.stack([loss_a, loss_b])

        weighted_loss = lambda_weights * loss_tasks
        if initial_loss is None:
            initial_loss = weighted_loss

        loss = weighted_loss.sum()

        # lambda_weights.retain_grad()
        loss_tasks.retain_grad()

        loss.backward(retain_graph=True)

        norms = []
        # FAILURE POINT
        for w_i, l_i in zip(lambda_weights, loss_tasks):
            local_grad = torch.autograd.grad(l_i, model.stage3.parameters(), retain_graph=True)[0]
            norms.append(torch.norm(w_i * local_grad))

        norms = torch.stack(norms)
        nw = norms.mean()
        with torch.no_grad():
            # loss ratios
            loss_ratios = loss_tasks / initial_loss
            # inverse training rate r(t)
            inverse_train_rates = loss_ratios / loss_ratios.mean()
            constant_term =  nw * (inverse_train_rates ** alpha)

        # compute Lgrad
        lgrad = (norms - constant_term).abs().sum()
        tgrad = torch.autograd.grad(lgrad, lambda_weights)
        lambda_weights.grad  = tgrad[0]

        model_optim.step()
        lambda_optim.step()

        # Renormalize
        with torch.no_grad():
            renormalize = num_tasks / lambda_weights.sum()
            lambda_weights *= renormalize

        weights_a_m.update(lambda_weights[0].item(), lambda_weights.size(0))
        weights_b_m.update(lambda_weights[1].item(), lambda_weights.size(0))
        loss_m.update(loss.item(), img.size(0))
        loss_a_m.update(loss_a.item(), img.size(0))
        loss_b_m.update(loss_b.item(), img.size(0))
        running_loss_a += loss_a.item()
        running_loss_b += loss_b.item()
        _, predicted = torch.max(out_a.data, 1)
        total_samples += target.size(0)
        correct_predictions += (predicted == target).sum().item()

    # Calculate average loss and accuracy for the epoch
    epoch_loss_a = running_loss_a / len(train_dataloader)
    epoch_loss_b = running_loss_b / len(train_dataloader)
    epoch_accuracy = correct_predictions / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss_a: {epoch_loss_a:.4f}, Loss_b: {epoch_loss_b:.4f}, W1: {weights_a_m.avg} W2: {weights_b_m.avg} Accuracy: {epoch_accuracy:.4f}')




RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.